# MUM 2023-24 Gradientowe uczenie sieci neuronowych

## Problemy
<img style="float: right;" src="ml_figures/sgd-convergence-divergence.gif" width=450>

1. sieci głębokie działają wolniej i mają więcej parametrów
2. silniejszy model - większy overfitting
3. wiele __minimów lokalnych__
  * model liniowy z wypukłą funkcją kosztu ma tylko jedno __minimum globalne__
    * podążając w kierunku przeciwnym do gradientu w końcu osiągniemy minimum
  * sieci neuronowe mają bardzo dużo minimów lokalnych
    * optymalizacja gradientowa nie umie wydostać się z minimum lokalnego
    * prawdopodobnie nigdy nie osiągniemy minimum globalnego
4. problemy z backpropagation w warstwowym modelu
  * warstwy uczą się z __różną prędkością__
  * np. głębsze warstwy uczą się bardzo powoli, a ostatnie warstwy bardzo szybko próbują rozwiązać problem
    * słabe wyniki
    * nie korzystamy w ogóle z głębokiej struktury
  * może być też zupełnie na odwrót
5. __vanishing gradient__
  * w algorytmie backpropagation delta musi przejść od najwyższej do najniższej warstwy
  * w każdej warstwie jest szansa, że kawałek gradientu zostanie (prawie) wyzerowany
    * np. jeśli funkcja aktywacji saturuje
    * albo dla ujemnych wartości w funkcji ReLU
  * im głębiej, tym bardziej efekty te się nawarstwiają
  * w najgłębszej warstwie może okazać się, że gradient jest bardzo mały
  * mały gradient oznacza, że warstwa ma mały wpływ na output sieci
    * dopiero wyższe warstwy mają istotny wpływ (większy gradient)
6. __exploding gradient__
  * w algorytmie backpropagation delta mnożona jest przez macierz wag (przypomnieć sobie wzór)
  * jeśli wagi są duże, to delta może zostać zwiększona
  * znowu, im głębiej, tym bardziej te efekty mogą się nawarstwiać
  * w najgłębszej warstwie gradient może być ogromny
    * całkowity brak stabilności uczenia
    * mała zmiana inputu zupełnie zmieni output sieci

## Różne możliwe rozwiązania
<img style="float: right;" src="ml_figures/sgd-training-rate.jpg" width=400>

### Znormalizowany input

1. transformacja zbioru treningowego
  * trzeba też transformować zbiór testowy!
2. cel - każda współrzędna inputu ma mieć średnią zero i wariancję jeden
  * estymujemy średnią i wariancję dla każdej kolumny zbioru treningowego
  * odejmujemy średnią, dzielimy przez pierwiastek z wariancji
  * zapamiętujemy wektor średnich i wektor wariancji, aby móc transformować przyszłe dane
  * __nie powinno się estymować tych parametrów przy użyciu zbioru testowego__
    * zbiór testowy ma symulować dane z przyszłości
    * nie możemy używać go podczas uczenia
    * normalizacja jest częścią procesu uczenia sieci
3. jeszcze lepiej działałaby __dekorelacja cech__
  * jeśli dwie kolumny cech są skorelowane, to niosą mniej informacji
  * w skrajnym przypadku dwie kolumny cech są identyczne
    * największa korelacja
  * dekorelacja wymaga obliczenia i zapamiętania macierzy kowariancji
    * kowariancja każdej pary cech
    * złożoność kwadratowa względem $D$ (obliczenie wariancji ma złożoność liniową)
    * zazwyczaj się tego nie robi

### Inicjalizacja wag sieci

1. inicjalizacja zerami utrudnia uczenie
2. trzeba inicjalizować losowymi wartościami
  * trzeba mądrze losować
3. [[Xavier Glorot, Yoshua Bengio, "Understanding the difficulty of training deep feedforward neural networks"]](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
  * kod initializer'a w notebooku na ćwiczeniach
4. pomysł polega na tym, aby (przynajmniej początkowo) wszystkie warstwy dostawały znormalizowany input
  * zaawansowane rozszerzenie - _batch normalization_

### Uczenie batchowe

1. liczymy gradient przy pomocy mini-batch'a (np. 64 przykłady treningowe)
2. niedokładne przybliżenie gradientu dla całego zbioru treningowego
  * szybsze
  * zmniejsza overfitting
    * przez dodatkową losowość model ma problem z zapamiętaniem zbioru treningowego
3. badania nad związkiem rozmiaru mini-batch'a z learning rate
  * zmniejszanie learning rate ma podobny efekt jak zwiększanie batch size
  * i na odwrót

### Regularyzacja

1. dodajemy do kosztu dodatkowy składnik
2. regularyzacja L2
  * suma kwadratów wszystkich parametrów ze wszystkich warstw sieci
  * przemnożona przez hiperparametr $\lambda$
3. regularyzacja ma za zadanie zniechęcić sieć do nauczenia się skrajnie dużych wartości dla niektórych parametrów